In [1]:
#!/usr/bin/env python3
import sys
import copy
sys.path.append('../')

from __future__ import print_function
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.theory import theoryPrediction
from smodels.experiment.databaseObj import Database, ExpResultList
from smodels.tools import coverage
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.printers.masterPrinter import MPrinter
from smodels.tools import ioObjects
import time
import os
from collections import OrderedDict
from configparser import ConfigParser, NoSectionError, NoOptionError
setLogLevel("error")

In [2]:
# Set the path to the database
database = Database(os.path.expanduser('../test/database'))
databaseVersion = database.databaseVersion

In [3]:
parameterFile='../test/testPrinters_parameters.ini'
parser = ConfigParser(inline_comment_prefixes=(';',))
ret = parser.read(parameterFile)
parser.set("printer","outputType","log")
parser.set("printer","outputFormat","current")

inputFile = '../test/testFiles/slha/lightEWinos.slha'
outputDir = '../test/'

In [4]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
minmassgap = parser.getfloat("parameters", "minmassgap") * GeV

masterPrinter = MPrinter()
masterPrinter.setPrinterOptions(parser)
masterPrinter.setOutPutFiles(os.path.join(
    outputDir, os.path.basename(inputFile)))

In [5]:
listOfExpRes = database.getExpResults()
masterPrinter.addObj(ExpResultList(listOfExpRes))

inputStatus = ioObjects.FileStatus()
if parser.getboolean("options", "checkInput"):
    inputStatus.checkFile(inputFile)

printParameters = []
if parser.has_section('parameters'):
    printParameters += list(parser.items('parameters'))
if parser.has_section('particles'):
    printParameters += list(parser.items('particles'))
if parser.has_section('options'):
    printParameters += list(parser.items('options'))

printParameters = OrderedDict(printParameters)
outputStatus = ioObjects.OutputStatus(inputStatus.status, inputFile,
                                      printParameters, databaseVersion)
masterPrinter.addObj(outputStatus)

In [6]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
promptWidth = None
stableWidth = None
if parser.has_option("particles", "promptWidth"):
    promptWidth = parser.getfloat("particles", "promptWidth")*GeV
if parser.has_option("particles", "stableWidth"):
    stableWidth = parser.getfloat("particles", "stableWidth")*GeV
model.updateParticles(inputFile=inputFile,
                      promptWidth=promptWidth, stableWidth=stableWidth)

In [7]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
smstoplist = decomposer.decompose(model, sigmacut,
                                  massCompress=parser.getboolean(
                                      "options", "doCompress"),
                                  invisibleCompress=parser.getboolean(
                                      "options", "doInvisible"),
                                  minmassgap=minmassgap)


In [8]:
masterPrinter.addObj(smstoplist)

In [9]:
allPredictions = []
combineResults = False
useBest = True
combineResults = parser.getboolean("options", "combineSRs")
allSRs = parser.getboolean("options", "reportAllSRs")
if allSRs:  # If set print out all SRs and skip combination
    useBest = False
    combineResults = False

for expResult in listOfExpRes:
    theorypredictions = theoryPredictionsFor(expResult, smstoplist,
                                             useBestDataset=useBest, combinedResults=combineResults,
                                             marginalize=False)
    if not theorypredictions:
        continue
    allPredictions += theorypredictions._theoryPredictions

if parser.getboolean("options", "computeStatistics"):
    for theoPred in allPredictions:
        theoPred.computeStatistics()

maxcond = parser.getfloat("parameters", "maxcond")
theoryPredictions = theoryPrediction.TheoryPredictionList(
    allPredictions, maxcond)


In [10]:
outputStatus.updateStatus(1)
masterPrinter.addObj(theoryPredictions)

In [11]:
if parser.getboolean("options", "testCoverage"):
    """ Testing coverage of model point, add results to the output file """
    if parser.has_option("options", "coverageSqrts"):
        sqrts = parser.getfloat("options", "coverageSqrts")*TeV
    else:
        sqrts = None
    uncovered = coverage.Uncovered(
        smstoplist, sigmacut=sigmacut, sqrts=sqrts)
    masterPrinter.addObj(uncovered)


In [12]:
if parser.has_option("options", "combineAnas"):
    """ Combine analyses """

    combineAnas = parser.get("options", "combineAnas").replace(" ","").split(",")
    if combineAnas:
        if combineResults is True:
            logger.warning("Combining analyses with signal region combination (combineSRs=True) might significantly reduce CPU performance.")
        combiner = TheoryPredictionsCombiner.selectResultsFrom(theoryPredictions,
                                                               combineAnas)
        # Only compute combination if at least one result was selected
        if combiner is not None:
            combiner.computeStatistics()
            masterPrinter.addObj(combiner)


In [13]:
masterCopy = copy.deepcopy(masterPrinter)
x = masterPrinter.flush()
with open(masterPrinter.Printers['log'].filename,'r') as f:
    for l in f.readlines():
        print(l)
os.remove(masterCopy.Printers['log'].filename)                

Input status: 1

Decomposition output status: 1 #decomposition was successful

# Input File: ../test/testFiles/slha/lightEWinos.slha

# checkinput = True

# combineanas = ATLAS-CONF-2013-037,CMS-SUS-13-012

# combinesrs = False

# computestatistics = True

# docompress = True

# doinvisible = True

# experimentalfeatures = False

# maxcond = 0.2

# minmassgap = 5.

# model = share.models.mssm

# ncpus = 1

# promptwidth = 1e-11

# reportallsrs = False

# sigmacut = 1.0

# stablewidth = 1e-25

# testcoverage = True

# Database version: unittest220



||                                                        ||

||             Selected Experimental Results              ||

||                                                        ||



Experimental Result ID: ATLAS-SUSY-2016-08

Tx Labels: ['T5Disp']

Sqrts: 1.30E+01

	 -----------------------------

	 SMS tested by analysis:

	    (PV > anyBSM(1),anyBSM(2)), (anyBSM(1) > q,q,MET), (anyBSM(2) > q,q,MET) 


Experimental Result ID: ATLAS-S

		 SMS ID: 44

		 SMS: (PV > C1-(1),N2(2)), (C1-(1) > N1~,e-,nu), (N2(2) > N1,b,b)

		 Masses: [(C1-, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.87E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.45E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 45

		 SMS: (PV > C1-(1),C1+(2)), (C1-(1) > N1~,mu-,nu), (C1+(2) > N1,nu,e+)

		 Masses: [(C1-, 1.34E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.41E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.29E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 46

		 SMS: (PV > C1-(1),C1+(2)), (C1-(1) > N1~,mu-,nu), (C1+(2) > N1,nu,mu+)

		 Masses: [(C1-, 1.34E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 	

 		 Sqrts: 1.30E+01 [TeV], Weight:3.57E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:5.38E-04 [pb]

 		 

		 .........................................................................

		 SMS ID: 600

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > t-,b,C1+(3)), (gluino(2) > g,N4(4)), (C1+(3) > N1,q,c), (N4(4) > W-,C1+(5)), (C1+(5) > N1,nu,mu+)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N4, 2.92E+02 [GeV]), (N1, 6.81E+01 [GeV]), (C1+, 1.34E+02 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:3.57E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:5.38E-04 [pb]

 		 

		 .........................................................................

		 SMS ID: 601

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > t-,b,C1+(3)), (gluino(2) > g,N4(4)), (C1+(3) > N1,q,c), (N4(4) > W-,C1+(5)), (C1+(5) > N1,nu,ta+)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N4, 2.92E+02 [GeV]), (N

In [14]:
masterCopy.outputFormat = "version2"
for printer in masterCopy.Printers.values():
    printer.outputFormat = "version2"

masterCopy.flush()    
with open(masterCopy.Printers['log'].filename,'r') as f:
    for l in f.readlines():
        print(l)
os.remove(masterCopy.Printers['log'].filename)        

Input status: 1

Decomposition output status: 1 #decomposition was successful

# Input File: ../test/testFiles/slha/lightEWinos.slha

# checkinput = True

# combineanas = ATLAS-CONF-2013-037,CMS-SUS-13-012

# combinesrs = False

# computestatistics = True

# docompress = True

# doinvisible = True

# experimentalfeatures = False

# maxcond = 0.2

# minmassgap = 5.

# model = share.models.mssm

# ncpus = 1

# promptwidth = 1e-11

# reportallsrs = False

# sigmacut = 1.0

# stablewidth = 1e-25

# testcoverage = True

# Database version: unittest220



||                                                        ||

||             Selected Experimental Results              ||

||                                                        ||



Experimental Result ID: ATLAS-SUSY-2016-08

Tx Labels: ['T5Disp']

Sqrts: 1.30E+01

	 -----------------------------

	 SMS tested by analysis:

	    [[[q,q]],[[q,q]]] 


Experimental Result ID: ATLAS-SUSY-2016-081

Tx Labels: ['T5Disp']

Sqrts: 1.30E+01

	


		 The element PIDs are 

		 PIDs: [[1000035, -1000024, -1000022], [1000037, 1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:1.00E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:5.73E-04 [pb]

 		 

		 .........................................................................

		 SMS: 

		 SMS ID: 295

		 Particles in element: [[[W+],[q,q]],[[Z],[q,c]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [2.92E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [2.93E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000035, -1000024, -1000022], [1000037, 1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:1.00E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:5.73E-04 [pb]

 		 

		 .........................................................................

		 SMS: 

		 SMS ID: 296

		 Particles in element: [[[W+],[q,c]],[[Z],[q,q]]]

		 Final states in element: [

 		 Sqrts: 1.30E+01 [TeV], Weight:4.24E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:6.38E-04 [pb]

 		 

		 .........................................................................

		 SMS: 

		 SMS ID: 556

		 Particles in element: [[[t-,b],[nu,ta+]],[[g],[W-],[q,c]]]

		 Final states in element: [N1,N1]

		 The element masses are 

		 Branch 0: [5.77E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [5.77E+02 [GeV], 2.66E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000021, 1000024, 1000022], [1000021, 1000025, 1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:4.24E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:6.38E-04 [pb]

 		 

		 .........................................................................

		 SMS: 

		 SMS ID: 557

		 Particles in element: [[[t-,b],[nu,ta+]],[[g],[W+],[q,q]]]

		 Final states in element: [N1,N1~]

		 The element masses are 

		 Branch 0: [5.77E+02 [GeV], 1.34E+02 [GeV], 6.8


		 Branch 0: [5.77E+02 [GeV], 1.35E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [5.77E+02 [GeV], 2.93E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000021, 1000023, 1000022], [1000021, -1000037, -1000024, -1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:5.44E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:8.20E-04 [pb]

 		 

		 .........................................................................

		 SMS: 

		 SMS ID: 829

		 Particles in element: [[[b,b],[b,b]],[[b,t+],[Z],[q,c]]]

		 Final states in element: [N1,N1~]

		 The element masses are 

		 Branch 0: [5.77E+02 [GeV], 1.35E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [5.77E+02 [GeV], 2.93E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000021, 1000023, 1000022], [1000021, -1000037, -1000024, -1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:5.44E-03 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:8.20E-04 [pb]

 		 